In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from tensorflow import keras as k

import tensorflow as tf
import pandas as pd
import numpy as np

# %pip install d2l==1.0.0a1.post0
from d2l import tensorflow as d2l


In [2]:
data = pd.read_csv('./input/dataset.csv')
# dataImputed = pd.read_csv('./input/datasetImputed.csv')

In [3]:
# è stata riscritta una funzione di reshape inutile 
# e.g. (a_prev=x_train, season=trn_ssn)
def reshape_to_inputshape(a_prev,season):
    totalMatches = len(season)*38
    input_step = int(a_prev.shape[0]/totalMatches)
    prev_f = a_prev.shape[1]
    return np.reshape(a_prev, (totalMatches, input_step, prev_f))

In [4]:
features = ['HomeTeam', 'AwayTeam', 
            'HTeamEloScore', 'ATeamEloScore', 
            'HTdaysSinceLastMatch', 'ATdaysSinceLastMatch', 
            'HTW_rate', 'ATW_rate', 'ATD_rate', 'HTD_rate', 
            '7_HTW_rate', '12_HTW_rate', '7_ATW_rate', '12_ATW_rate', 
            '7_HTD_rate', '12_HTD_rate', '7_ATD_rate', '12_ATD_rate',
            '7_HTL_rate', '12_HTL_rate', '7_ATL_rate', '12_ATL_rate',
            '5_HTHW_rate', '5_ATAW_rate']

X = pd.get_dummies(data[features])

# Se non cambiamo nulla il OneHotEncoder assegna:
# A -> 1 0 0
# D -> 0 1 0
# H -> 0 0 1
y = data[['FTR']].to_numpy().ravel().reshape(-1, 1)
y = OneHotEncoder(sparse=False).fit_transform(y)
X_imputed = SimpleImputer().fit_transform(X)



In [5]:
trn_ssn = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
trn_ssn_len = len(trn_ssn)
tst_ssn = [2016,2017,2018] 
tst_ssn_len = len(tst_ssn)

test_size = float(tst_ssn_len)/(tst_ssn_len+trn_ssn_len)

#Split X and Y into training and Test Sets
x_train, x_test, y_train, y_test = train_test_split(X_imputed, y, shuffle=False, test_size=test_size)

In [6]:
def time_step(a_prev,season):
    a_prev = a_prev[np.newaxis, ...]
    totalMatches = len(season)*38

    prev_f = a_prev.shape[2]
    input_step = int(a_prev.shape[1]/totalMatches)
    step = 0
    a_new = np.zeros((totalMatches, input_step, prev_f))
    for i in range(totalMatches):
        # rows divise in porzioni di totalMatches rows
        step += input_step
        
        # per tutte le righe nell'intervallo di righe che stiamo guardando ora 
        # va in ogni porzione di righe di volta in volta
        for j in range(step-input_step,step):

            # per ogni colonna
            for k in range(prev_f):
                a_new[i, j - input_step * i, k] = a_prev[:, j, k]
    
    return a_new

In [7]:
#Setup XY to have 10 game steps
lstm_x_train = reshape_to_inputshape(x_train,trn_ssn)
lstm_y_train = reshape_to_inputshape(y_train,trn_ssn)
# y_train = np.moveaxis(y_train, 0, 1)

lstm_x_test = reshape_to_inputshape(x_test,tst_ssn)
lstm_y_test = reshape_to_inputshape(y_test,tst_ssn)
# y_test = np.moveaxis(y_test, 0, 1)

Tx = lstm_x_train.shape[1] #Time steps
Ty = lstm_y_train.shape[0] #Time Steps

num_features = lstm_x_train.shape[2] #Features per step
inputs = tf.keras.Input(shape=(Tx, num_features))

In [8]:
print(lstm_x_train.shape)
print(lstm_y_train.shape)

(304, 10, 94)
(304, 10, 3)


# Prova di tensorflow

In [9]:
inputs = tf.random.normal((32, 10, 8))
lstm = tf.keras.layers.LSTM(18)
output = lstm(inputs)
print(output.shape)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-11-09 16:39:46.248972: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-09 16:39:46.249420: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


(32, 18)


In [10]:
inputs = np.array([[[0, 1], [0, 1]], [[1, 2], [1, 2]], [[2, 3], [2, 3]]], dtype=np.float64)
lstm = k.models.Sequential([
    tf.keras.layers.LSTM(2)
])
out = lstm(inputs)
print(out)


tf.Tensor(
[[-0.12013333 -0.20862854]
 [-0.04657596 -0.32306018]
 [-0.01559365 -0.37989762]], shape=(3, 2), dtype=float32)


In [11]:
MLP = tf.keras.models.Sequential([
    k.layers.Dense(4000),
    k.layers.Dropout(.7),
    k.layers.Dense(100),
    k.layers.Dense(3, activation='softmax')
])

In [12]:
MLP.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=[tf.keras.metrics.Accuracy()]
)
MLP(x_train)
MLP.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (3040, 4000)              380000    
                                                                 
 dropout (Dropout)           (3040, 4000)              0         
                                                                 
 dense_1 (Dense)             (3040, 100)               400100    
                                                                 
 dense_2 (Dense)             (3040, 3)                 303       
                                                                 
Total params: 780,403
Trainable params: 780,403
Non-trainable params: 0
_________________________________________________________________


In [13]:
MLP.fit(lstm_x_train, lstm_y_train)

2022-11-09 16:39:47.248862: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2022-11-09 16:39:47.596115: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 1s 25ms/step - loss: 19.2480 - accuracy: 0.1394


# RandomForestClassifieri

In [14]:
forest = RandomForestClassifier(n_estimators=2, random_state=2)
forest = forest.fit(x_train, y_train)


#Forest Model Metrics
print("Forest Classifier")
print("Train Score: ", forest.score(x_train, y_train))
print("Test Score: ", forest.score(x_test, y_test))

Forest Classifier
Train Score:  0.6302631578947369
Test Score:  0.22017543859649122


In [15]:
n = 10
m = 5
max_depth = 10
forests = []
grid = [{"n_estimators": list(range(1, n)), "random_state": list(range(0, m)), "max_depth": list(range(1, max_depth))}]
gridSearch = GridSearchCV(RandomForestClassifier(), param_grid=grid, n_jobs=10, return_train_score=True)
gridSearch.fit(x_train, y_train)
# for i in range(1, n+1):
#     for j in range(m):
#         forest = RandomForestClassifier(n_estimators=i, random_state=j)
#         forest = forest.fit(x_train, y_train)
#         forests.append(forest)
#         print("Forest Classifier, n_estimators: ", i, "random_state: ", j)
#         print("Train Score: ", forest.score(x_train, y_train))
#         print("Test Score: ", forest.score(x_test, y_test))

# #Forest Model Metrics
# print("Forest Classifier")
# print("Train Score: ", forest.score(x_train, y_train))
# print("Test Score: ", forest.score(x_test, y_test))
print("Forest Classifiers Best Score: ", gridSearch.best_score_)
print("Forest Classifiers Best Params: ", gridSearch.best_params_)
print("Forest Classifiers Best Params: ", gridSearch.best_estimator_)

Forest Classifiers Best Score:  0.4305921052631579
Forest Classifiers Best Params:  {'max_depth': 8, 'n_estimators': 1, 'random_state': 1}
Forest Classifiers Best Params:  RandomForestClassifier(max_depth=8, n_estimators=1, random_state=1)


# LSTM

In [16]:
x_prova = reshape_to_inputshape(x_train, trn_ssn)
y_prova = reshape_to_inputshape(y_train, trn_ssn)
x_prova_test = reshape_to_inputshape(x_test, tst_ssn)
y_prova_test = reshape_to_inputshape(y_test, tst_ssn)

In [103]:
# 65-67% Accuracy no Overfit
model = k.models.Sequential(
    [
        k.layers.GRU(64, return_sequences=True),
        k.layers.Dropout(0.4),
        k.layers.Dense(1000, activation="relu"),
        k.layers.Dropout(0.3),
        k.layers.Dense(250, activation="relu"),
        k.layers.Dropout(0.2),
        k.layers.Dense(3, activation="softmax")
    ]
)

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

model.fit(x_prova, y_prova, epochs=500, callbacks=[callback])

Epoch 1/500


2022-11-09 18:18:08.880714: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-09 18:18:09.051425: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-09 18:18:09.556831: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 3s 140ms/step - loss: 1.0802 - accuracy: 0.4224
Epoch 2/500
10/10 [==============================] - 1s 60ms/step - loss: 1.0327 - accuracy: 0.4928
Epoch 3/500
10/10 [==============================] - 0s 39ms/step - loss: 1.0140 - accuracy: 0.5188
Epoch 4/500
10/10 [==============================] - 0s 35ms/step - loss: 1.0103 - accuracy: 0.5214
Epoch 5/500
10/10 [==============================] - 0s 44ms/step - loss: 0.9946 - accuracy: 0.5355
Epoch 6/500
10/10 [==============================] - 0s 36ms/step - loss: 0.9907 - accuracy: 0.5214
Epoch 7/500
10/10 [==============================] - 0s 39ms/step - loss: 0.9875 - accuracy: 0.5303
Epoch 8/500
10/10 [==============================] - 0s 39ms/step - loss: 0.9852 - accuracy: 0.5247
Epoch 9/500
10/10 [==============================] - 0s 31ms/step - loss: 0.9794 - accuracy: 0.5322
Epoch 10/500
10/10 [==============================] - 0s 32ms/step - loss: 0.9746 - accuracy: 0.5401
Epoch 11/5

In [104]:
model.evaluate(x_prova_test, y_prova_test)

2022-11-09 18:18:30.512321: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-09 18:18:30.591054: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 1s 93ms/step - loss: 0.9210 - accuracy: 0.5833


[0.9210423827171326, 0.5833333730697632]

In [ ]:
print(x_prova_test.shape)
result = model.predict(x_prova_test)
print(result)

In [105]:
def revert_yoh(Y):
    Y_new = np.empty([Y.shape[0],Y.shape[1]], dtype="<U1")
    #Y_new = np.zeros((Y.shape[0],Y.shape[1]))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if (Y[i, j] == 0):
                Y_new[i, j]= 'A'
            elif (Y[i, j] == 1):
                Y_new[i, j]= 'D'
            elif (Y[i, j] == 2):
                Y_new[i, j]='H'
    return Y_new

y_pred = model.predict(x_prova_test)
y_predm = np.asarray(y_pred)
y_predm = np.argmax(y_predm, axis=2)
y_testm = np.argmax(y_prova_test, axis=2)

y_pred_train = model.predict(x_prova)
y_pred_train = np.asarray(y_pred_train)
y_predm_train = np.argmax(y_pred_train, axis=2)
y_trainm = np.argmax(y_prova, axis = 2)

y_predm = revert_yoh(y_predm).ravel()
y_testm = revert_yoh(y_testm).ravel()

y_predm_train = revert_yoh(y_predm_train).ravel()
y_trainm = revert_yoh(y_trainm).ravel()

#Model Metrics
print(classification_report(y_testm, y_predm, digits=3))

2022-11-09 18:18:33.964463: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-09 18:18:34.019677: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 0s 21ms/step
              precision    recall  f1-score   support

           A      0.576     0.614     0.595       345
           D      0.287     0.161     0.207       254
           H      0.655     0.762     0.704       541

    accuracy                          0.583      1140
   macro avg      0.506     0.512     0.502      1140
weighted avg      0.549     0.583     0.560      1140



# Neural Network

In [ ]:
y_nn = ((data[['ordinalHR']]).to_numpy()*2)
#Split X and Y into training and Test Sets
y_nn = OneHotEncoder(sparse=True).fit_transform(y_nn)
x_train, x_test, y_train, y_test = train_test_split(X_imputed, y_nn, shuffle=True)

In [ ]:
nn = k.models.Sequential([
    k.layers.Input(shape=[x_train.shape[1]]),
    k.layers.Flatten(),
    k.layers.Dense(41, activation='relu'),
    k.layers.Dense(75, activation='relu'),
    k.layers.Dropout(0.3),
    k.layers.Dense(3, activation='softmax'),
])

In [ ]:
learning_rate=0.01

nn.compile(
    loss='categorical_crossentropy',
    optimizer=k.optimizers.Adam(learning_rate),
    metrics=['categorical_accuracy']
)
nn.summary()

In [ ]:
epochs=1000
batch_size=50
nn.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

In [ ]:
print(nn.evaluate(x_test, y_test))